# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг. Откроем файл с данными и изучим общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Открыли файл с данными и просмотрели информацию по каждому столбцу

In [2]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Иучили как выглядит сама таблица на примере первых 15 строк.

**Вывод**

По таблице видно, что необходима предобработка данных. В столбце 'education' использованы разные регистры и группировать по этому столбцу не получится. Также имеются пропуски с которыми в последующем нужно разобраться. Также в столбце 'family_status_id' данные типа object, думаю стоит их перевести в данные типа int. Этим сейчас и займемся

## Шаг. Предобработка данных

### Обработка пропусков

Попробую заполнить пропуски средним значением по категориям 'income_type'.

In [3]:
df[(df['total_income'].isnull() == True) & (df['days_employed'].isnull() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [4]:
#переведу все значения days_employed в положительные
df['days_employed'] = df['days_employed'].abs()
df_grouped = df.groupby('income_type').agg({'days_employed':['count', 'mean', lambda x: sum(x>0)]})

df_to_rename = dict(zip(df_grouped.columns.levels[1], ['Общее количество строк', 'Среднее', 'Кол-во значений > 0']))

df_grouped = df_grouped.rename(columns=df_to_rename, level=1)
df_grouped

days_employed                                   
                Общее количество строк        Среднее Кол-во значений > 0
income_type                                                              
безработный                          2  366413.652744                 2.0
в декрете                            1    3296.759962                 1.0
госслужащий                       1312    3399.896902              1312.0
компаньон                         4577    2111.524398              4577.0
пенсионер                         3443  365003.491245              3443.0
предприниматель                      1     520.848083                 1.0
сотрудник                        10014    2326.499216             10014.0
студент                              1     578.751554                 1.0

In [5]:
df['ratio_days_employed'] = df[df['days_employed'].notnull()]['days_employed']/((df['dob_years']-16)*365)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,ratio_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


In [6]:
#-'days_employed': NaN заменяем на произведение среднего коэффициента по группе 'income_type', умноженного на возраст в днях;
#-'total_income': NaN заменяем на среднее значение по зарплате по группе 'income_type'.
df['days_employed'] = df.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()*df['dob_years']*365))
df['total_income'] = df.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,ratio_days_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.889112
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.551343
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,0.906273
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.706292
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,25.195563


In [7]:
#Удаляю вспомогательный столбец и вывожу общую информацию по датафрейму.
df = df.drop('ratio_days_employed', axis=1)
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8.437673e+03,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4.024804e+03,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5.623423e+03,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4.124747e+03,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,3.402661e+05,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,9.261858e+02,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2.879202e+03,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,1.527796e+02,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6.929865e+03,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2.188756e+03,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Заполнили пропуски, теперь пенсионеры получают как и другие пенсионеры и все в таком духе. Я понял свою ошибку

### Замена типов данных

In [8]:
#перевод значений в 'days_employed' в положительные, предположительно произошла опечатка в записи этих данных.
#также перевод ['total_income'] и ['days_employed'] значений типа 'float' в целочисленные 'int'
#еще были обнаружены отрицательные значения в столбы 'children'
df['children'] = df['children'].abs()
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

После перевода в целочисленный тип данных столбцов со стажем работы и ежемесячным доходом сразу таблица стала более читабельной.

### Обработка дубликатов

In [9]:
#были обнаружены знания с разным регистром в толбце 'education', что может привести к неправильному поиску дупликатов
#поэтому сначала привели к общему регистру, а затем посчитали количество дупликатов
df['education'] = df['education'].str.lower()
df.duplicated().sum()

71

In [10]:
#и удалили их
df = df.drop_duplicates()
df.duplicated().sum()

0

**Вывод**

Были удалены дупликаты с учетом регистра, что поможет нам при подведении итогов проекта

### Лемматизация

In [11]:
from pymystem3 import Mystem
m = Mystem()
 
def lemmatize_purpose(row):
    
    lemmas = m.lemmatize(row)
    for word in lemmas:
        if word in ['недвижимость', 'жилье']:
            return 'недвижимость'
        if word == 'автомобиль':
            return 'автомобиль'
        if word == 'образование':
            return 'образование'
        if word == 'свадьба':
            return 'свадьба'
        
        
df['category'] = df['purpose'].apply(lemmatize_purpose)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

Лемматизировать столбец с целями кредита нужно, чтобы выделить основные цели из разных вариантов их описания клиентами.
Для начала я подключил библиотеку Mystem и затем создал функцию которая лемматизирует данные, а потом распределяет по категориям кредита. 

### Категоризация данных

In [12]:
# использую метод qcut для нахождения интервалов и проделаем разбиение по категориям
df['quantile_ex_1'] = pd.qcut(df['total_income'], q=4)
df['quantile_ex_1'].unique()

[(202417.0, 2265604.0], (107623.0, 151887.0], (151887.0, 202417.0], (20666.999, 107623.0]]
Categories (4, interval[float64]): [(20666.999, 107623.0] < (107623.0, 151887.0] < (151887.0, 202417.0] < (202417.0, 2265604.0]]

In [13]:
#Сделаю категоризацию по возрасту людей
def age_group(age):   
    if age <= 18:
        return 'молодой'
    if age <= 64:
        return 'взрослый'
    return 'пенсионер' 
df['age_group'] = df['dob_years'].apply(age_group)


def money_they_earn(money):   
    if (money >= 20666.999 and money < 107623.0):
        return 'малый доход'
    elif (money >= 107623.0 and money < 151887.0):
        return 'средний доход'
    elif (money >= 151887.0 and money < 202417.0):
        return 'высокий доход' 
    return 'богатый'
df['money_they_earn'] = df['total_income'].apply(money_they_earn)
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category,quantile_ex_1,age_group,money_they_earn
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,"(202417.0, 2265604.0]",взрослый,богатый
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,"(107623.0, 151887.0]",взрослый,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,"(107623.0, 151887.0]",взрослый,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,"(202417.0, 2265604.0]",взрослый,богатый
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,"(151887.0, 202417.0]",взрослый,высокий доход
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,"(202417.0, 2265604.0]",взрослый,богатый
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,"(202417.0, 2265604.0]",взрослый,богатый
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,"(107623.0, 151887.0]",взрослый,средний доход
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,"(20666.999, 107623.0]",взрослый,малый доход
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,"(107623.0, 151887.0]",взрослый,средний доход


In [14]:
#Необходимо проверить корректность всех пар 'education'-'education_id' и 'family_status'-'family_status_id', 
#то есть те данные, которым уже присвоены категории
df.groupby(['education', 'education_id']).size().to_frame('count').reset_index()

,education,education_id,count
0,высшее,0,5250
1,начальное,3,282
2,неоконченное высшее,2,744
3,среднее,1,15172
4,ученая степень,4,6


In [15]:
df.groupby(['family_status', 'family_status_id']).size().to_frame('count').reset_index()

,family_status,family_status_id,count
0,Не женат / не замужем,4,2810
1,в разводе,3,1195
2,вдовец / вдова,2,959
3,гражданский брак,1,4151
4,женат / замужем,0,12339


Итог: пары уникальны.

**Вывод**

Я сделал категоризацию данных по детям в семье(нет детей, малодетные, многодетные) и возрасту(молодой, взрослый, пенсионер) клиентов. 

## Шаг. Ответим на интересующие нас вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [16]:
#Функция для вывода информации
def relation(category):
    return df.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')

relation('children')

,debt
children,
5,0.000000
0,0.075438
3,0.081818
1,0.091658
2,0.094542
4,0.097561
20,0.105263


In [17]:
df['children'].value_counts()

0     14091
1      4855
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

**Вывод**

Заемщики без наличия детей менее склонны к просрочке по выплатам кредита. Случаи когда есть 5, 4 и 20 детей рассматривать нельзя, так как слишком мало случаев. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [18]:
df2 = df.pivot_table(index=['debt'], columns='family_status', values='age_group', aggfunc='count')
no_family = df2['Не женат / не замужем'][1] / df2['Не женат / не замужем'][0]
divorce = df2['в разводе'][1] / df2['в разводе'][0]
widow = df2['вдовец / вдова'][1] / df2['вдовец / вдова'][0]
partner = df2['гражданский брак'][1] / df2['гражданский брак'][0]
family = df2['женат / замужем'][1] / df2['женат / замужем'][0]
print("{0:.2f}% Не женат / не замужем".format(no_family*100))
print("{0:.2f}% в разводе".format(divorce*100))
print("{0:.2f}% вдовец / вдова".format(widow*100))
print("{0:.2f}% гражданский брак".format(partner*100))
print("{0:.2f}% женат / замужем".format(family*100))

10.80% Не женат / не замужем
7.66% в разводе
7.03% вдовец / вдова
10.31% гражданский брак
8.16% женат / замужем


**Вывод**

Если человек не женат/не замужем или в гражданском браке то он лучше выплачивает кредит

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [19]:
df3 = df.pivot_table(index=['debt'], columns='money_they_earn', values='age_group', aggfunc='count')

in1 = df3['богатый'][1] / df3['богатый'][0]
in2 = df3['высокий доход'][1] / df3['высокий доход'][0]
in3 = df3['средний доход'][1] / df3['средний доход'][0]
in4 = df3['малый доход'][1] / df3['малый доход'][0]


print("{0:.2f}% богатый".format(in1*100))
print("{0:.2f}% высокий доход".format(in2*100))
print("{0:.2f}% средний доход".format(in3*100))
print("{0:.2f}% малый доход".format(in3*100))


7.45% богатый
9.56% высокий доход
9.72% средний доход
9.72% малый доход


**Вывод**

люди с малым количеством дохода и средним чаще всего не возвращают кредит в срок

- Как разные цели кредита влияют на его возврат в срок?

In [20]:
#использую функцию, которую написал для ответа на первый вопрос
relation('category')

,debt
category,
недвижимость,0.072334
свадьба,0.080034
образование,0.092200
автомобиль,0.093590


**Вывод**

Люди с большей вероятностью не выплачивают кредит если он взять на операции с автомобилем или образованием. 

## Шаг. Общий вывод

Я провел исследование, выполнил предобработку данных: заменил пропуски сдредним значением по группам должностей людей, заменил вещественный тип данных на целочисленный, удалил дупликаты, провел лемматизацию данных и категоризировал данные по количеству дохода и по возрасту. 
Отвечая на поставленный вопрос, могу дать следующий ответ - цель кредита и количество детей влияет на факт погашения кредита в строк:
-заемщики с целями на автомобиль и образование, при этом имеющие 1 или 2 детей - самые менее ответственные заемщики;
-заемщики, с остальными целями, и не имеющие детей - самые ответственные заемщики.